In [6]:
# Setup
import sys
from pathlib import Path

# Add the project root to the path
project_root = Path("..").resolve()
sys.path.insert(0, str(project_root))

from data.embedder import build_vector_index, load_lore_json
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.llms import MockLLM
import chromadb

In [ ]:
# Load and build index (if not done already)
lore_data = load_lore_json(Path("data/lore.json"))
index = build_vector_index(lore_data)

In [ ]:
# Load from persisted DB
vector_store = ChromaVectorStore(persist_dir="db/minecraft_lore")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

In [ ]:
# Query test
query_engine = index.as_query_engine()
response = query_engine.query("What is a creeper?")
print(str(response))